# Simple simulation in NS-3 using Python bindings

First, let's import the bindings of NS-3.

In [1]:
from ns import ns

Let's start with simple two-node topology. Nodes are connected via P2P link with specified datarate and delay.

![topo](./fig/NS3-p2p-layers.drawio.png)

## Create two nodes

Nodes can be created in various ways. The most straightforward one is to use the NodeContainer's Create method.

In [2]:
nodes = ns.network.NodeContainer()
nodes.Create(2)

As we are using Python bindings that are autogenerated by cppyy, the class name of an instance has a cppyy prefix.

In [3]:
nodes

The more information about the Nodes and NodeContainer can be found in NS-3 documentation:
- https://www.nsnam.org/docs/release/3.39/doxygen/d7/db2/classns3_1_1_node_container.html
- https://www.nsnam.org/docs/release/3.39/doxygen/d0/d3d/classns3_1_1_node.html

## Link them together using network devices

Nodes are empty shells without any devices (network cards) for as long as you do not install one or more. This can be achieved using the helper classes of various link types. The most basic one is the P2P link.

In [4]:
p2p = ns.point_to_point.PointToPointHelper()
p2p.SetDeviceAttribute("DataRate", ns.core.StringValue("5Mbps"))
p2p.SetChannelAttribute("Delay", ns.core.StringValue("2ms"))

devices = p2p.Install(nodes)

Now, devices is actually an instance of NetDeviceContainer class, so it wraps the devices that have common features. In this example, they are the two ends of P2P link. More comprehensive information about NetDeviceContainer can be found in NS-3 documentation:
- https://www.nsnam.org/docs/release/3.39/doxygen/d6/ddf/classns3_1_1_net_device_container.html
- https://www.nsnam.org/docs/release/3.39/doxygen/d0/da3/classns3_1_1_net_device.html


In [5]:
devices

To iterate over the devices in container, you can use the GetN method to get the number of devices and then iterate using Python's range built-in function.

In [6]:
devices.GetN()

2

Individual devices have the attributes set, so you can actually get the information about them.

### Task
1. Using the NS-3 documentation, get the MAC addresses of both NetDevice instances.
2. Distinguish what device is on what node.

### Discussion
What format does MAC address have?

## Add IP Stack

Now we have two nodes that are linked together. On both we must install IP stack to get L3 functionality up and running. We can again use a helper class that will take care of the process.

In [7]:
ip_stack = ns.internet.InternetStackHelper()
ip_stack.Install(nodes)

In [8]:
ip_stack

The actual adressing can be achieved either automatically or manually. Here is the automated approach.

In [9]:
addr = ns.internet.Ipv4AddressHelper()
addr.SetBase(ns.network.Ipv4Address("10.0.0.0"),
                ns.network.Ipv4Mask("255.255.255.0"))

ifaces = addr.Assign(devices)

### Task
1. Create an interface manually and assign its address manually as well.

In [10]:
ifaces

By assigning and IP address to the device, we are creating an interface that is used for the communication with the rest of the simulation topology. In this, NS-3 terminology more resembles virtual interfaces rather than actual ones. The documentation for Ipv4InterfaceContainer and Ipv4Interface can be found here:
- https://www.nsnam.org/docs/release/3.39/doxygen/db/d86/classns3_1_1_ipv4_interface_container.html
- https://www.nsnam.org/docs/release/3.39/doxygen/dc/da9/classns3_1_1_ipv4_interface.html

As with any other container, Ipv4InterfaceContainer can be used to obtain a specific interface.

In [11]:
ifaces.Get(0)

<cppyy.gbl.std.pair<ns3::Ptr<ns3::Ipv4>,unsigned int> object at 0x55d5f70d3b50>

### Task
1. Iterate over interfaces and read their addresses.

## Add Application together with Transport layer functions

One of the possible applications that can be used with NS-3 simulator is UdpEchoServer/Client. To create these, we can use helpers that take care of creating appropriate transport part of the socket for the user. Again the result of the operation is a container, in this case it is ApplicationContainer.
- https://www.nsnam.org/docs/release/3.39/doxygen/df/dbe/classns3_1_1_application_container.html
- https://www.nsnam.org/docs/release/3.39/doxygen/de/d96/classns3_1_1_application.html
- https://www.nsnam.org/docs/release/3.39/doxygen/d5/d2f/classns3_1_1_udp_echo_client_helper.html
- https://www.nsnam.org/docs/release/3.39/doxygen/d6/d64/classns3_1_1_udp_echo_server_helper.html

In [12]:
ECHO_PORT = 9
echo_srv_helper = ns.UdpEchoServerHelper(ECHO_PORT)
srv_apps = echo_srv_helper.Install(nodes.Get(1))
print(f"Servers: {srv_apps.GetN()}")

Servers: 1


In [13]:
srv_apps

In [14]:
srv_addr = ifaces.GetAddress(1).ConvertTo()
print(f"Server IP address: {ifaces.GetAddress(1)}")

Server IP address: 10.0.0.2


In [15]:
print(srv_addr)

03-04-0a:00:00:02


In [16]:
srv_addr

srv_addr is an internal representation of an IP Address of the server. It is an instance of generic Address class that is used internally by the NS-3 simulator and this one is needed by the client applications.

### Task
1. Create a UdpEchoClientHelper object and set attributes for the client, such as how many packets will be sent, inter-packet intervals and packet size.

In [17]:
# Change it here!!!
echo_client_helper = None

A helper snippet to get actually set params of the client application:

In [20]:
client_apps = echo_client_helper.Install(nodes.Get(0))
print(f"Client apps: {client_apps.GetN()}")

attrs = {
    "MaxPackets": ns.UintegerValue,
    "Interval": ns.TimeValue,
    "PacketSize": ns.UintegerValue,
}
for attr, f in attrs.items():
    f = f()
    client_apps.Get(0).__deref__().GetAttribute(attr, f)
    print(f"\t{attr}: {f.Get()}")

Client apps: 1
	MaxPackets: 50
	Interval: +1e+09ns
	PacketSize: 1500


## Time the simulation

In [21]:
srv_apps.Start(ns.core.Seconds(1.0))
srv_apps.Stop(ns.core.Seconds(10.0))
client_apps.Start(ns.core.Seconds(2.0)) # has to be later than server
client_apps.Stop(ns.core.Seconds(10.0))

## Run the simulator

In [23]:
ns.Simulator.Stop(ns.Seconds(10))
ns.Simulator.Run()
ns.Simulator.Destroy()

At time +2s client sent 1500 bytes to 10.0.0.2 port 9
At time +2.00448s server received 1500 bytes from 10.0.0.1 port 49153
At time +2.00448s server sent 1500 bytes to 10.0.0.1 port 49153
At time +2.00897s client received 1500 bytes from 10.0.0.2 port 9
At time +3s client sent 1500 bytes to 10.0.0.2 port 9
At time +3.00448s server received 1500 bytes from 10.0.0.1 port 49153
At time +3.00448s server sent 1500 bytes to 10.0.0.1 port 49153
At time +3.00897s client received 1500 bytes from 10.0.0.2 port 9
At time +4s client sent 1500 bytes to 10.0.0.2 port 9
At time +4.00448s server received 1500 bytes from 10.0.0.1 port 49153
At time +4.00448s server sent 1500 bytes to 10.0.0.1 port 49153
At time +4.00897s client received 1500 bytes from 10.0.0.2 port 9
At time +5s client sent 1500 bytes to 10.0.0.2 port 9
At time +5.00448s server received 1500 bytes from 10.0.0.1 port 49153
At time +5.00448s server sent 1500 bytes to 10.0.0.1 port 49153
At time +5.00897s client received 1500 bytes from 

### Task
1. Add basic logging to the application.
2. Modify the application so that it uses OnOff Application.
3. Get the routing table for the given nodes.
4. Expand the example to a more elaborate topology - ie. star.